# ***Load data***

In [1]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import files

uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [2]:
import pandas as pd
import io

# Extract filename
filename = list(uploaded.keys())[0]

#    Load CSV from either:
#      - a filename (string path)
#      - an uploaded file from Colab's files.upload() dict
#      - a Flask file object (from request.files)

if isinstance(uploaded[filename], str):
  # Assume it's a file path
  df = pd.read_csv(uploaded[filename])
elif hasattr(uploaded[filename], 'read'):
  # Flask's file object or BytesIO
  df = pd.read_csv(io.BytesIO(uploaded[filename].read()))
elif isinstance(uploaded[filename], bytes):
  # Bytes directly (Colab uploaded dict value)
  df = pd.read_csv(io.BytesIO(uploaded[filename]))
else:
  raise ValueError("Unsupported file source type.")



# ***Set up x & y, scale x, and one-hot encode categorical features***

In [6]:
from sklearn.preprocessing import StandardScaler


# Use all numeric features to train model
x = df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited', ])
y = df['Exited']

cat_features_to_encode = ['Geography', 'Gender']
x = pd.get_dummies(x, columns=cat_features_to_encode, drop_first=False)

# Scale the data
x = StandardScaler().fit_transform(x)

# ***Split the data for training and testing***

In [7]:
from sklearn.model_selection import train_test_split

# Set aside 20% for for testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=5)

# 20% of train for validataion, same as test size
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=8)


# ***Create the ANN Deep Learning model***

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

ann = Sequential()

# 3 layers, 128 and 64 filters for training
ann.add(Input(shape=(x_train.shape[1],)))
ann.add(Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
ann.add(Dense(64, activation='relu'))
ann.add(Dense(1, activation='sigmoid'))  # sigmoid for binary classification

# Compile the model, using the binary crossentropy as the target is binary
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# ***Fit, train, and evaluate the model***

In [17]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks in case performance stops improving
early_stop = EarlyStopping(
    monitor='val_loss',       # what to monitor
    patience=5,               # epochs to wait before stopping
    restore_best_weights=True # roll back to best weights
)

# Define checkpoint to save the best model weights
checkpoint = ModelCheckpoint(
    'best_model.keras',       # filename to save best model
    monitor='val_loss',
    save_best_only=True       # save only the best one
)

history = ann.fit(x_train, y_train, epochs=50, batch_size=128,
                  validation_data=(x_val, y_val), callbacks=[early_stop, checkpoint])

# Load the best saved model
best_model = load_model('best_model.keras')

# Use it for prediction
y_pred = best_model.predict(x_test)

loss, accuracy = ann.evaluate(x_test, y_test)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8945 - loss: 0.2543 - val_accuracy: 0.8344 - val_loss: 0.3889
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8914 - loss: 0.2623 - val_accuracy: 0.8469 - val_loss: 0.3724
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8947 - loss: 0.2493 - val_accuracy: 0.8487 - val_loss: 0.3763
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9003 - loss: 0.2448 - val_accuracy: 0.8525 - val_loss: 0.3731
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8950 - loss: 0.2471 - val_accuracy: 0.8394 - val_loss: 0.3881
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8953 - loss: 0.2466 - val_accuracy: 0.8556 - val_loss: 0.3801
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8929 - loss: 0.2482 - val_accuracy: 0.8462 - val_loss: 0.3855
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8285 - loss: 0.4096


# ***Make predictions***

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Get probabilities from ANN
predictions = ann.predict(x_test)

# Convert probabilities to class labels
# Binary classification threshold = 0.5
y_pred = (predictions > 0.5).astype(int)

# Confusion matrix and accuracy score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

print("\nAccuracy Score:", acc)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Confusion Matrix:
[[1475  120]
 [ 215  190]]

Accuracy Score: 0.8325
